# Notebook to analyse Results from CNN Training

In [18]:
# Load needed packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as pyplot

In [25]:
# Import result data
res_dat = pd.read_excel("/Users/maxbehrens/Documents/Msc/Thesis/Data/results/cnn_results_cleaned.xlsx", skiprows=1)
res_dat.loc[res_dat['lr']==1,'lr'] = 0.001
res_dat.tail(10)

,date,no_layers,batch_size,epochs,res,dims,train_mse,train_mae,val_mse,val_mae,test_mse,test_mae,transfer,lr,optim,comment,type,prediction
83,2020-03-29,181,16,46,256,3,1.000001,0.402500,0.210931,0.342692,3.743913e+07,4623.755444,resnet_model,0.000001,adam,iterated nn,viirs_night,nuts_diff
84,2020-03-29,181,16,107,256,3,1.000503,0.398351,0.206448,0.337493,3.647298e+07,4493.468813,resnet_model,0.001000,rmsprob,iterated nn,viirs_night,nuts_diff
85,2020-03-29,181,16,41,256,3,1.000369,0.397534,0.205430,0.336254,3.650967e+07,4498.403978,resnet_model,0.000100,rmsprob,iterated nn,viirs_night,nuts_diff
86,2020-03-29,181,16,54,256,3,1.000332,0.397619,0.205539,0.336388,3.650858e+07,4498.258503,resnet_model,0.000010,rmsprob,iterated nn,viirs_night,nuts_diff
87,2020-03-29,181,16,45,256,3,1.000331,0.397606,0.205523,0.336368,3.650848e+07,4498.245052,resnet_model,0.000001,rmsprob,iterated nn,viirs_night,nuts_diff
88,2020-03-29,181,16,49,256,3,1.000020,0.401923,0.210250,0.341917,3.694366e+07,4557.916217,resnet_model,0.001000,adamax,iterated nn,viirs_night,nuts_diff
89,2020-03-29,181,16,39,256,3,1.000080,0.400253,0.208462,0.339867,3.695893e+07,4559.988459,resnet_model,0.000100,adamax,iterated nn,viirs_night,nuts_diff
90,2020-03-29,181,16,66,256,3,1.000091,0.400043,0.208222,0.339589,3.696407e+07,4560.683876,resnet_model,0.000010,adamax,iterated nn,viirs_night,nuts_diff
91,2020-03-29,181,16,63,256,3,1.000091,0.400025,0.208206,0.339570,3.696476e+07,4560.777753,resnet_model,0.000001,adamax,iterated nn,viirs_night,nuts_diff
92,2020-03-29,3,16,92,256,3,1.000401,0.621457,0.403024,0.511362,1.633803e+08,10984.113821,xs_model,0.001000,sgd,iterated nn,viirs_night,nuts_value


# Filter by Validation MSE

## Optimising Function

Adam seems to work best.

In [20]:
res_dat.groupby(['prediction','optim'])['val_mse'].mean().reset_index().sort_values('val_mse')

,prediction,optim,val_mse
2,nuts_diff,rmsprob,0.205196
1,nuts_diff,adamax,0.209564
3,nuts_diff,sgd,0.225406
0,nuts_diff,adam,0.281097
4,nuts_value,sgd,0.403024


## Model Type

- For nuts_value smaller models and transfer learning seems to lead to better results.
- For nuts_diff transfer models and larger models seems to work better

In [21]:
res_dat.groupby(['prediction','transfer'])['val_mse'].mean().reset_index().sort_values(['val_mse'])

,prediction,transfer,val_mse
5,nuts_diff,xs_model,0.208664
3,nuts_diff,resnet_model,0.209401
1,nuts_diff,m_model,0.209402
0,nuts_diff,inception_model,0.215212
2,nuts_diff,mobile_model,0.222514
4,nuts_diff,s_model,0.347134
6,nuts_value,xs_model,0.403024


## Learning Rate

Smaller learning rate seems to improve the results.

In [22]:
res_dat.groupby(['prediction','lr'])['val_mse'].mean().reset_index().sort_values(['val_mse'])

,prediction,lr,val_mse
2,nuts_diff,0.000100,0.215653
0,nuts_diff,0.000001,0.225966
1,nuts_diff,0.000010,0.226620
3,nuts_diff,0.001000,0.253879
4,nuts_value,0.001000,0.403024


## Batch Size

Batch size seems not to make a difference.

In [23]:
res_dat.groupby(['prediction','batch_size'])['val_mse'].mean().reset_index().sort_values(['val_mse'])

,prediction,batch_size,val_mse
0,nuts_diff,8,0.208664
1,nuts_diff,16,0.235132
2,nuts_value,16,0.403024


## Putting it all together

In [26]:
res_dat.groupby(['prediction','transfer','lr'])['val_mse'].mean().reset_index().sort_values(['val_mse'])

,prediction,transfer,lr,val_mse
1,nuts_diff,inception_model,0.000010,0.208105
22,nuts_diff,xs_model,0.000100,0.208131
21,nuts_diff,xs_model,0.000010,0.208373
20,nuts_diff,xs_model,0.000001,0.208387
14,nuts_diff,resnet_model,0.000100,0.209067
12,nuts_diff,resnet_model,0.000001,0.209091
13,nuts_diff,resnet_model,0.000010,0.209100
7,nuts_diff,m_model,0.001000,0.209162
6,nuts_diff,m_model,0.000100,0.209456
5,nuts_diff,m_model,0.000010,0.209475


- The best model seems to come from RMSProp Optimiser, very small model and high learning rate

In [28]:
res_dat.groupby(['prediction','optim','transfer','lr'])['val_mse'].mean().reset_index().sort_values(['val_mse'])

,prediction,optim,transfer,lr,val_mse
71,nuts_diff,rmsprob,xs_model,0.001000,0.200480
70,nuts_diff,rmsprob,xs_model,0.000100,0.202079
69,nuts_diff,rmsprob,xs_model,0.000010,0.202300
68,nuts_diff,rmsprob,xs_model,0.000001,0.202375
64,nuts_diff,rmsprob,s_model,0.000001,0.204975
...,...,...,...,...,...
18,nuts_diff,adam,s_model,0.000100,0.311689
92,nuts_value,sgd,xs_model,0.001000,0.403024
16,nuts_diff,adam,s_model,0.000001,0.502132
17,nuts_diff,adam,s_model,0.000010,0.546015


In [35]:
res_dat.loc[res_dat['val_mse']==min(res_dat['val_mse'])]

,date,no_layers,batch_size,epochs,res,dims,train_mse,train_mae,val_mse,val_mae,test_mse,test_mae,transfer,lr,optim,comment,type,prediction
8,2020-03-27,3,8,69,256,3,1.001475,0.393669,0.20048,0.330197,3.608354e+07,4439.712597,xs_model,0.001,rmsprob,iterated nn,viirs_night,nuts_diff
